# LIGHT GBM RESGESYON

In [1]:
!pip install lightgbm

In [2]:
# Yazılım dünyasında yeni çıkan herşey yüzde 90 daha iyi değildir. Yeni çıkan şeylerinde bir eskimesini beklemek gerekir.

In [3]:
# Yukarıdaki kurulumda hata olursa - conda için
# conda install -c conda-forge lightgbm

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn import model_selection

In [2]:
df = pd.read_csv("verisetleri\Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state=42)

In [3]:
import lightgbm

In [4]:
from lightgbm import LGBMRegressor

In [7]:
lgb_model = LGBMRegressor().fit(X_train, y_train)

In [9]:
lgb_model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [10]:
y_pred = lgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

363.8712087611089

## MODEL TUNNING

In [25]:
lgb_model = LGBMRegressor()

In [28]:
lgb_model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [18]:
# PARAMETRELER NASIL SEÇİLİR

# Öntanımlı parametrelere bakarak, buradaki değerin etrafındaki değerler denenebilir.

lgbm_params = {"learning_rate" : [0.01, 0.1, 0.5, 1], "n_estimators" : [20, 40, 100, 1000], "max_depth" : [1, 2, 3, 4, 5, 6, 7 ,8 ,9, 10] }

In [29]:
lgbm_cv_model = GridSearchCV(lgb_model, lgbm_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 160 candidates, totalling 1600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 1172 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 1600 out of 1600 | elapsed:   29.4s finished


In [30]:
lgbm_cv_model.best_params_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 20}

In [31]:
lgbm_tuned = LGBMRegressor(learning_rate=0.1, max_depth=6, n_estimators=20).fit(X_train, y_train)

In [32]:
y_pred = lgbm_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

371.5044868943621